<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/PDF%E3%82%92%E5%85%83%E3%81%ABQ%26A%EF%BC%88LLM_LangChain%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://qiita.com/hiroki_okuhata_int/items/7102bab7d96eb2574e7d
# https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

In [ ]:
# 機械学習品質マネジメントガイドラインのPDFを使用
# https://www.digiarc.aist.go.jp/publication/aiqm/AIQuality-requirements-rev3.2.1.0079-signed.pdf

In [ ]:
%%writefile requirements.txt
openai
chromadb
langchain
pypdf
tiktoken

In [ ]:
%pip install -r requirements.txt

In [4]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader

In [5]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 秘密情報を記載したファイルの場所を指定
envfile_mount = '/content/drive/MyDrive/Colab Notebooks/env/'

# 秘密情報が保存されているテキストファイルのパス
secrets_file_path = os.path.join(envfile_mount, 'openaiapikey.txt')

# テキストファイルから秘密情報を読み込む
with open(secrets_file_path, 'r') as f:
    api_key = f.read().strip()

In [47]:
# データセットのファイルの場所を指定
dataset_mount = '/content/drive/My Drive/Colab Notebooks/data_set/'

# ターゲットのPDF
target_pdf = 'AIQuality-requirements-rev3.2.1.0079-signed.pdf'

# データセットが保存されているテキストファイルのパス
data_file_path = os.path.join(dataset_mount, target_pdf)

# データセットを読み込む
loader = PyPDFLoader(data_file_path)
pages = loader.load_and_split()

In [8]:
# 状況確認
pages[10].page_content

'機械学習品質マネジメントガイドライン 第3版  国立研究開発法人産業技術総合研究所  \nDigiARC-TR-2022 -05 / CPSEC -TR-2022 006 \n1 \n 1. ガイドライン全体 概要 \n 本章 の内容は参考 （i nformative ）である。 本章 に含まれ、 本ガイドラインの規範の一部を\n構成する (normative) 内容については、後の 章で再掲する。  \n \n 本概要の全体構成は以下の通りである。第 2章以降の本編に対応する内容がある場合に\nは、その対応する章 節も示す。  \n・ 1.1節では、本ガイドラインを作成した背景と目的を示す。  \n・ 1.2節では、本ガイドラインが主に想定する「使われ方」を提示する。 （本編 2章）  \n・ 1.3節では、 背景として掲げた「 AIの品質管理が困難である理由」を分析する。  \n・ 1.4節では、本ガイドラインがベースとする、品質管理のプロセスについての全体的\nな考え方を述べる。  \n・ 1.5節では、 本ガイドラインが 「目標」 として設定する５つ の 「外部品質」 の観点 （実\n装手段にあまり依存せず、 使用を通じてしか評価できない品質観点） を提示する。 （本\n編3章）  \n・ 1.6節では前節を補足して、一般に「 AIの品質」として議論される要素のうち、前節\nで採用しなかった要素について、その理由や本ガイドラインでの考え方を説明する。  \n・ 1.7節では、本ガイドラインが「管理手段」として設定する 9つの「内部品質」の観\n点（ 実 装 手段に依存し、 計測や作り込みでの管理が可能な品質観点） を提示する。 （本\n編6章・ 7章） \n・ 1.8節では、本ガイドラインを作成するに当たって想定する開発プロセスの全体のイ\nメージを提示する（本編 4章・5章）  \n・ 1.9節では、外部の規範文書などとの関係を明らかにする。 （本編 11章） \n・ 1.10節では、本ガイドラインの残りの部分の構成を整理する。  \n1.1 目的と背景 \n 人工知能（ AI） 、とりわけ機械学習技術は、製造業、自動運転、ロボット、ヘルスケア、\n金融、 小売 などの幅広い応用分野で有効性が確認され、 社会実装が本格化する兆しを見せて\nいる。その一方

In [22]:
# API KEYを設定する
openai.api_key = api_key

# 使うモデルを設定
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [23]:
# PDFをベクトル化
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectorstore.persist()

In [24]:
# Q&Aの設定
pdf_qa = ConversationalRetrievalChain.from_llm(llm,
                                               vectorstore.as_retriever(),
                                               return_source_documents=True,
                                               max_tokens_limit=4097
                                               )

In [48]:
# TEST
query = "What is this report? Answer within 10 words in Japanese."
chat_history = []
result = pdf_qa({"question": query, "chat_history": chat_history})
result["answer"]

'機械学習品質マネジメントガイドライン第3版の報告書。'

In [49]:
result['source_documents'][0]

Document(page_content='機械学習品質マネジメントガイドライン 第3版  国立研究開発法人産業技術総合研究所  \nDigiARC-TR-2022- 05 / CPSEC -TR-2022 006 \n149 \n \n 開発基盤・ツールの事例  \n8.6.2.1  典型的な機能・モジュール \n前節に述べた趣旨から望まれる主な機能群を以下にあげる。  \n① データセットの可視化（要配慮属性や関連の発見のための分析）  \n② メトリクス以外の公平性確認（CBN で述べたように特定のデータ（データポイン\nト）に対する Counterfactual 等、一部の属性を現実と変えた場合の実験など） \n③ 様々な公平性メトリクス計測  \n④ XAI ライブラリー（決定に、どの属性が寄与したかの可視化）  \n⑤ 運用後の継続的モニタリング＆再学習といったライフサイクルを支援するパイプ\nライン構築基盤（いわゆる DevOps 基盤＋データセット、モデルに関する基盤）  \n以下に Google とIBM Fairness360 を参考に紹介するが、 他の各種ベンダーからの新機能\nも継続的に提供されるため、最新の情報を確認のうえ、最適なものを選択されたい。  \n8.6.2.2  Google のツール例 \n前節で述べた機能①～⑤が、以下のようにカバーされる。  \nWhat -If-tool    ①、② \nhttps://pair -code.github.io/what -if-tool/learn/tutorials/walkthrough/  \nFairness Indicators  ③ \nExplainable AI    ④ \nhttps://cloud.google.com/explainable- ai/ \nAI-Platform     ⑤ \nhttps://cloud.google.com/ai- platform  \n公平性に直結する多彩な機能を提供する What -If-tool は単にデータセットの可視化では\nなく、 仮想シナリオや、 様々な属性でスライスした場合の分析機能などを通じて、 データセ\nットに潜在的に存在するバイアスパターンの発見に役立つ。  \nまた、Explainable AI 

In [62]:
query = "機械学習の品質維持に必要な案件を簡潔にまとめてください。"
chat_history = []

# search distance に閾値を設定
vectordbkwargs = {"search_distance": 0.9}
# q&a
result = pdf_qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
# 結果表示
print(result["answer"])
print("参照頁")
print([d['page'] for d in [i.metadata for i in result['source_documents']]])

機械学習の品質維持に必要な案件は以下の通りです。

1. 問題領域分析の十分性を確保すること。
2. 運用開始時点で充足されていた内部品質を、運用期間中を通じて維持するための技術を確立すること。
3. 外部環境の変化に対応するため、機械学習要素を変化に対して追従させる必要があること。
4. モニタリング技術を導入し、データ分布が時間経過に伴い変化する現象を検知すること。
5. 再学習のための技術と、追加の学習データの作成技術を確立すること。
参照頁
[31, 31, 128]


In [68]:
def get_answer(query):
  chat_history = []
  # search distance に閾値を設定
  vectordbkwargs = {"search_distance": 0.9}
  # q&a
  result = pdf_qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
  # 結果表示
  print(result["answer"])
  print("参照頁")
  print([d['page'] for d in [i.metadata for i in result['source_documents']]])

query = "品質維持ができる学習データとは？"
get_answer(query)

機械学習において、品質維持ができる学習データとは、以下の特性を持つデータのことを指します。

1. 均一性：データセット内の各状況や各ケースが、入力されるデータ全体におけるそれらの発生頻度に応じて抽出されていること。
2. 被覆性：学習モデルが対象とする実世界の特徴の子細について、要求分析から実装までの少なくともいずれかの段階で把握され、最終的にプログラム内の条件分岐や計算式などとして反映されること。
3. 多様性：データセット内には、対象とする実世界の特徴の多様なパターンが含まれていること。
4. 偏りのなさ：特定の状況やケースに偏りがなく、学習不足や偏ったデータによる特定の状況への学習漏れが起こらないこと。

これらの特性を持つ学習データを用いることで、機械学習モデルの品質を維持することができます。
参照頁
[33, 33, 175, 175]


In [69]:
query = "What kind of training data can maintain quality?"
get_answer(query)

According to the context provided, there are several factors that can affect the quality of training data, and maintaining quality depends on various factors such as the nature of the problem, the content of individual data, risk aversion, and the priority of external quality of AI performance. Therefore, it is important to have clear policies or decision-making processes in place to determine whether rare data that deviates from the trend should be adopted as meaningful training targets or rejected as outliers or measurement errors. Additionally, physical security and diversity of the data acquisition environment should be ensured, and quality assurance activities should be recorded. Finally, the use of up-to-date data and the implementation of a system for monitoring quality are also important for maintaining quality.
参照頁
[96, 96, 111, 111]


In [71]:
query = "データ設計でやってはいけないことは記載されていますか？"
get_answer(query)

この文書には、データ設計でやってはいけないことについて明示的に記載されているわけではありません。ただし、データセットの改変を抑制・防止するための対策や、データセットの被覆性や均一性、ラベリングのポリシーの統一・精査、データセットの整合性チェック・再チェック、ロングテールの扱いや計測ミス・外れ値の判断、データ汚染への対応、最新性など、データセットの品質を確保するためのセキュリティ対策が記載されています。
参照頁
[204, 204, 225, 225]
